In [1]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

C:\anaconda\envs\tf1.14\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# class that defines and loads the kangaroo dataset
class frontBumperDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        self.add_class("dataset", 1, "frontBumper")
        # define data locations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annotations/'
        # find all images
        image_id = 0
        for filename in listdir(images_dir):
            image_id += 1
            img_path = images_dir + filename
            ann_path = annotations_dir + str(image_id) + '.xml'
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
			
 
	# extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
            # load and parse the file
        tree = ElementTree.parse(filename)
                # get the root of the document
        root = tree.getroot()
                # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
                # extract image dimensions
        width = 450
        height = 380
        return boxes, width, height
 
	# load the masks for an image
    def load_mask(self, image_id):
    # get details of image
        info = self.image_info[image_id]
                # define box file location
        path = info['annotation']
                # load XML
        boxes, w, h = self.extract_boxes(path)
                # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
                # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('frontBumper'))
        return masks, asarray(class_ids, dtype='int32')
 
        # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [3]:
# define a configuration for the model
class frontBumperConfig(Config):
	# Give the configuration a recognizable name
	NAME = "frontBumper_cfg"
	# Number of classes (background + parts)
	NUM_CLASSES = 2
	# Number of training steps per epoch #number of training images
	STEPS_PER_EPOCH = 40
 
# prepare config
config = frontBumperConfig()

In [4]:
# prepare train set
train_set = frontBumperDataset()
train_set.load_dataset('E:/4emeS2/PIDS/defective_parts/', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

Train: 50


In [5]:
# prepare test/val set
test_set = frontBumperDataset()
test_set.load_dataset('E:/4emeS2/PIDS/defective_parts/', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Test: 50


In [6]:
# prepare config
config = frontBumperConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [7]:
# define the model
model = MaskRCNN(mode='training', model_dir='', config=config)

In [8]:
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [11]:
model.compile(learning_rate=config.LEARNING_RATE, momentum=0.1)

In [12]:
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: frontbumper_cfg20210429T1425\mask_rcnn_frontbumper_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4     

KeyboardInterrupt: 

In [ ]:
# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "frontBumper_cfg"
	NUM_CLASSES = 2

In [ ]:
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='', config=cfg)